In [ ]:
cap = cv2.VideoCapture("./data/videos/video1.mp4")
count = 0
while True:
    ret, frame = cap.read()
    if count >= 29970:
        if ret and (count <=29970 + 170):
            name = "./data/obj_train_data/images/frame_0"+str(count) + ".jpg"
            cv2.imwrite(name,frame)
        else:
            break
    
    count = count + 1
cap.release()

In [ ]:
frame = cv2.imread("./data/obj_train_data/images/frame_030138.jpg")

box = []
with open("./data/obj_train_data/labels/frame_030138.txt", "r") as f:
    for i in f:
        box.append(i.split())
f.close()

cl,x,y,w,h = [eval(i) for i in box[1]]
h_frame,w_frame,channel = frame.shape
# top left
xtl = int(x*w_frame-w*w_frame/2)
ytl = int(y*h_frame+h*h_frame/2)

# bottom right
xbr = int(x*w_frame+w*w_frame/2)
ybr = int(y*h_frame-h*h_frame/2)

img = cv2.rectangle(frame.copy(), (xtl,ytl),(xbr,ybr),(0,255,0),2 )
plt.imshow(img)

In [ ]:
# create train an valid txt
count = 0
for name in os.listdir("./data/images"):
    img_path = os.path.join(os.getcwd()+"/images/",name)
    if count < 120: #45 to valid
        with open("./train.txt","a") as x:
            x.write(img_path + "\n")
    else:
        with open("./valid.txt","a") as x:
            x.write(img_path + "\n")
    count = count + 1


In [ ]:
# train yolov7
!ls

In [ ]:
%cd yolov7

In [ ]:
!python train.py --workers 8 --device 0 --batch-size 10 --data ../data.yaml --img 1080 1920 --cfg cfg/training/yolov7.yaml --weight "../yolov7_training.pt" --name yolov7-custom --epoch 10